## Apply all imports here

In [2]:
from os import listdir, getcwd
import os
from os.path import isfile, join
import subprocess
import sys
import csv
import pandas as pd



## Read all csv files 

In [ ]:
mypath = join(getcwd(), 'eval_results/soot_res')
# print(mypath)

csv_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
# print(csv_files)

completepaths = [os.path.join(mypath, f) for f in csv_files]
# print(completepaths)

# desired_col = ["jar","solver","thread","totalRuntime","cgConstructionTime","prop","method","mem","CallGraphAlgo","callGraphEdges","callGraphReachableNodes","initialStmtCount","stmtCountAfterApplyingBI","BodyTransformers","BodyTransformersMetrics"]
desired_col = ["jar","solver","thread","totalRuntime","cgConstructionTime","prop","method","mem","CallGraphAlgo","callGraphEdges","callGraphReachableNodes","initialStmtCount","stmtCountAfterApplyingBI","BodyTransformers","BodyTransformersMetrics"]

for completefilepath in completepaths:
    # print(completefilepath)
    csv_cols = next(csv.reader(open(completefilepath))) # read only the csv columns names

    cols = [col for col in desired_col if col in csv_cols]
    df = pd.read_csv(completefilepath, sep=";", header=0)
    print(df.head())
    print("-----------")

In [ ]:
df.iloc[0]

## Convert csv with , files

In [70]:
import re
import json
import pandas as pd
import csv
import ast

mypath = join(getcwd(), 'eval_results/soot_res')
# print(mypath)

csv_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
# print(csv_files)

completepaths = [os.path.join(mypath, f) for f in csv_files]
# print(completepaths)

for completepath in completepaths:
    with open(completepath, 'r', encoding='UTF-8') as file:
        lines = [line.rstrip() for line in file]

print(lines[1])

# input_str = """
# jar,solver,thread,totalRuntime,cgConstructionTime,prop,method,mem,CallGraphAlgo,callGraphEdges,callGraphReachableNodes,initialStmtCount,stmtCountAfterApplyingBI,BodyTransformers,BodyTransformersMetrics
# assertj-core-3.24.2,default,1,88422,5368,3974427,50,554,CHA,57910,13599,101912,102790,[JB_LS, JB_TR],{jb.tr=[1470, -366], jb.ls=[903, -50]}
# assertj-core-3.24.2,default,1,62110,6085,3020975,50,594,CHA,57910,13599,101912,102790,[JB_LS, JB_TR, JB_LP],{jb.tr=[1424, 2], jb.lp=[697, -126], jb.ls=[731, -476]}
# """
input_str = '\n'.join(lines)
# print(input_str)

new_ouput = ""
new_different_output = ""
for each_row in input_str.split('\n'): 
    # print(each_row)
    if each_row.strip() != "":
        split_row = each_row.split(',')
        new_ouput += ','.join(split_row[:13])
        new_ouput += '\n'
        new_different_output += ','.join(split_row[13:])
        new_different_output += '\n'

with open('simple_output.csv', 'w') as file:
    file.write(new_ouput)

df = pd.read_csv('simple_output.csv')
# print(df.head())


pattern = r'^(\[.*?\]),(\{.*\})$'

each_index = 0
for each_row in new_different_output.split('\n'):
    if each_row.strip() != "" and not each_row.startswith('BodyTransformers'):
        match = re.match(pattern, each_row)
        if match:
            BodyTransformers_str = match.group(1)
            column_y = BodyTransformers_str.strip('[]').split(',')
            # print(each_index)
            df.loc[each_index, 'BodyTransformers'] = json.dumps(column_y)
            column_z_str = match.group(2).replace('=', ':')
            # column_z_str = re.sub(r'([\w\.]+):', r'"\1":', column_z_str)
            column_z_str = re.sub(r'([a-zA-Z0-9._-]+):', r'"\1":', column_z_str)
            # string_literal = repr(column_z_str)
            # print(column_z_str)
            column_z = json.loads(column_z_str)
            df.loc[each_index, 'BodyTransformersMetrics'] = json.dumps(column_z)
        each_index += 1

print(df.head())

commons-codec-1.15,default,1,8692,1575,398587,50,203,RTA,5907,1506,27940,24432,[JB_LS, JB_TR],{jb.tr=[347, 9], jb.ls=[206, -23]}
                  jar   solver  thread  totalRuntime  cgConstructionTime  \
0  commons-codec-1.15  default       1          8692                1575   
1  commons-codec-1.15  default       1          6227                1705   
2  commons-codec-1.15  default       1          8755                1524   
3  commons-codec-1.15  default       1          8994                1604   
4  commons-codec-1.15  default       1          8670                1660   

     prop  method  mem CallGraphAlgo  callGraphEdges  callGraphReachableNodes  \
0  398587      50  203           RTA            5907                     1506   
1  242702      50  171           RTA            5907                     1506   
2  398587      50  286           RTA            5907                     1506   
3  398000      50  260           RTA            5907                     1506   
4  397806

In [69]:
for key, value in df['BodyTransformersMetrics'].items():
    print(value)

{"jb.tr": [347, 9], "jb.ls": [206, -23]}
{"jb.tr": [383, 9], "jb.lp": [178, 13], "jb.ls": [204, 6]}
{"jb.tr": [320, -46], "jb.ese": [4, 0], "jb.ls": [198, 8]}
{"jb.tr": [362, -45], "jb.ls": [184, 7], "jb.ne": [5, 0]}
{"jb.tr": [347, -30], "jb.ls": [219, 6], "jb.dae": [121, 1]}
{"jb.ule": [17, 0], "jb.tr": [361, -47], "jb.ls": [215, 8]}
{"jb.cp": [97, 2], "jb.tr": [356, 7], "jb.ls": [205, -44]}
{"jb.tr": [346, -45], "jb.uce": [125, 1], "jb.ls": [156, 7]}
{"jb.tt": [58, 1], "jb.tr": [356, 4], "jb.ls": [175, -47]}
{"jb.lns": [3, 0], "jb.tr": [347, 6], "jb.ls": [245, -80]}
{"jb.tr": [386, 4], "jb.cbf": [9, 0], "jb.ls": [194, 8]}
{"jb.dtr": [1, 0], "jb.tr": [349, -53], "jb.ls": [187, 6]}
{"jb.tr": [259, 0], "jb.ls": [171, -54], "jb.sils": [646, -43]}
{"jb.tr": [285, 1], "jb.a": [208, 16], "jb.ls": [249, -52]}
{"jb.tr": [334, 4], "jb.ls": [202, 7], "jb.ulp": [210, -93]}
{"jb.tr": [392, -48], "jb.cp-ule": [15, 0], "jb.ls": [189, 7]}
{"jb.tr": [343, 2], "jb.ls": [303, -95]}
{"jb.tr": [404, -88